# Zadanie dnia

# Zadanie 1

* Należy pobrać z wykorzystaniem pakietów requests, Beautiful Soup wszystkie wydarzenia zamieszczone w harmonogramie na stronie https://www.mimuw.edu.pl/. 
* Proszę napisać scraper w taki sposób, aby w efekcie działania zapisywał informacje jako lista słowników, których kluczami są tytuł, url do wydarzenia oraz jego data i godzina.
* Otrzymane dane proszę zapisać w formacie csv/xlsx - pomocne może być wykorzystanie ramki danych (pakiet pandas) i metody from_dict.
* Na koniec - prosze o wyświetlenie pięciu losowych wydarzeń z listy pobranych (tytuł, url, data).

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
driver = webdriver.Chrome()
driver.get("https://www.mimuw.edu.pl/en/")
driver.implicitly_wait(0.5)
# Można teraz dokonywać interakcji ze stroną, np. klikając w przyciski, wypełniając formularze itp.
# Źródło strony
#print(driver.page_source)
def extract_from_doctorate(html):
    date = "no date"
    Title= "no title"
    link= "no link"
    room= "no room"
    speaker= "no speaker"
    soup = BeautifulSoup(html, 'html.parser')
    tooltip = soup.find('div', class_='calendar-tooltip')
    event_elements = tooltip.find_all('tr')
    header = tooltip.find('div', class_='calendar-tooltip-header').text
    if not event_elements:
        return "No events found."
    for info in event_elements:
        label = info.find('td', class_='calendar-tooltip-label')
        text = info.find('td', class_='calendar-tooltip-text')
        # print(label.text, text.text)
        if label and text:
            if(label.text == "From"):
                date = text.text
            elif(label.text == "Title" or label.text == "Thesis"):
                Title = text.text
                linked = text.find('a')
                if linked:
                    link = linked['href']
                    if not link.startswith('https'):
                        link = 'https://mimuw.edu.pl' + link
            elif(label.text == "Link"):
                link = text.find('a')['href']
            elif(label.text == "Room"):
                room = text.text
            elif(label.text == "Speaker(s)" or label.text == "Phd Student"):
                speaker = text.text
    return {'Name': header, 'Title': Title,'Date': date, 'Speaking': speaker, 'Link': link, 'Room': room}                



calendar_events = driver.find_elements(By.CLASS_NAME, 'calendar-event')
#print(calendar_events)
if calendar_events:
    events_list = []
    print("Found the following calendar events:")
    for index, event in enumerate(calendar_events, start=1):
        full = event.get_attribute("outerHTML")
        row = extract_from_doctorate(full)
        events_list.append(row)
    df = pd.DataFrame(events_list)

    random_sample = df.sample(5)
    random_sample.to_csv('calendar_events.csv', index=False)
    # Tutaj nie znalazłem lepszego sposobu na wyświetlanie tablicy.
    print(random_sample)
else:
    print("No calendar events found.")

Found the following calendar events:
               Name                                              Title  \
14  Talk on seminar                                                      
12  Talk on seminar  Normalized solutions to elliptic problems - ne...   
8   Talk on seminar     Diversity and evolution of the economic system   
7   Talk on seminar  A Euclidean Embedding for Computing Persistent...   
5   Talk on seminar               Bregman variation of semimartingales   

                Date             Speaking  \
14  2024-11-20 10:30     Kasia Jankiewicz   
12  2024-11-21 12:30  Bartosz Bieganowski   
8   2024-11-20 14:15        Elżbieta Pliś   
7   2024-11-22 14:15          Kunal Dutta   
5   2024-11-21 12:15        Dominik Kutek   

                                                 Link  Room  
14  https://mimuw.edu.pl/en/seminars/talk_pl/wlasn...  4070  
12  https://mimuw.edu.pl/en/seminars/talk/normaliz...  5070  
8   https://mimuw.edu.pl/en/seminars/talk/diversit...  5070  

# Zadanie 2

Należy wybrać dowolną stronę internetową (zgodnie z zainteresowaniami), która zezwala na pobieranie danych. Następnie proszę dokonać pobierania wybranych przez Państwa treści. Można spróbować wykorzystac Selenium, chociaż requests + Beautiful Soup są wystarczające. Uzasadnić do czego takie dane mogą się Państwu przydać.  